# Example 06-03: Sampling from a Poisson P(6) target

In [ ]:
using Distributions
using Gadfly
using Lora

In [ ]:
plogtarget(p::Int, v::Vector) = p*log(v[1])-lfact(p)

In [ ]:
p = BasicDiscUnvParameter(:p, logtarget=plogtarget, nkeys=2)

In [ ]:
model = likelihood_model([Constant(:λ), p], isindexed=false)

In [ ]:
psetproposal(i::Int) = (i == 0) ? Binary(0, 1) : Binary(i-1, i+1)

In [ ]:
sampler = MH(psetproposal, symmetric=false)

In [ ]:
mcrange = BasicMCRange(nsteps=10000, burnin=1000)

In [ ]:
v0 = Dict(:λ=>6, :p=>2)

In [ ]:
job = BasicMCJob(model, sampler, mcrange, v0)

In [ ]:
run(job)

In [ ]:
chain = output(job)

In [ ]:
mean(chain)

In [ ]:
acceptance(chain, diagnostics=false)

In [ ]:
xrange = 0.001:0.001:15
mcplot = plot(
  Guide.title("Poisson(6)"),
  layer(
    x=collect(xrange),
    y=[pdf(Poisson(6), v) for v in xrange],
    Geom.line,
    Theme(default_color=colorant"red")
  ),
  layer(x=chain.value, Geom.histogram(density=true))
)

In [ ]:
draw(PDF("/Users/theodore/tmp/poisson_hist.pdf", 8inch, 6inch), mcplot)